In [ ]:
#this part for colab
# !git clone https://github.com/REU-DS-CLUB/Face_id_and_detection/
# import os
# os.chdir('/content/Face_id_and_detection')
# !pip install -q comet-ml facenet-pytorch timm # для гугл колаба не нужно скачивать все из requirements.txt

import torch
import torch.nn as nn
from facenet_pytorch import InceptionResnetV1
import timm

import os
import json
from tqdm import tqdm
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

import src.models as mdls
import src.utils as utils

# Face recognition imports
from src.models import Triplet
from src.utils import TripletLoss


#downloading datasets
config = utils.get_options()


if config['use_colab']:
    utils.colab()
else:
    utils.check_if_datasets_are_downloaded()

import src.dataloaders as dataloaders



In [ ]:
img_size = config['img_size']

# Задаем директорию для checkpoints
checkpoint_dir = "checkpoints/"

# Проверяем, есть ли уже такая директори, если нет, создаем
os.makedirs(checkpoint_dir, exist_ok=True)

#определяем, доступен ли cude
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Face detection setup

# Задаем размер тестовой выборки для разделения данных на обучение и тест
test_size = 0.1

# Задаем скорость обучения для оптимизатора
det_lr = 1e-3

# Создаем экземпляр модели для обнаружения лиц (InspectorGadjet предполагается, что это название модели)
detection_model = mdls.InspectorGadjet()

# Получаем загрузчики данных для обучения и тестирования модели
train_detection_dataloader, test_detection_dataloader = dataloaders.get_train_test_dataloaders(dataloaders.dataset, test_size=test_size)

# Задаем функцию потерь для обучения модели (в данном случае Mean Squared Error Loss)
loss_fn = nn.MSELoss()

# Задаем оптимизатор (Adam) для обновления параметров модели
optimizer = torch.optim.Adam(detection_model.parameters(), lr=det_lr)

# Перемещаем модель на устройство (например, GPU, если доступен)
detection_model.to(device)

# Задаем шедулер для управления скоростью обучения (уменьшение скорости обучения на каждом шаге)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Задаем ранний остановщик для прекращения обучения, если потери не уменьшаются в течение некоторого времени
early_stopper = utils.EarlyStopping(patience=10, min_delta=0.01)

# Выводим количество элементов в обучающем загрузчике данных
print(len(train_detection_dataloader))


In [ ]:
### Function to check how albumentation works
# batch = next(iter(train_detection_dataloader))
# utils.plot_images_with_bboxes(batch)

In [ ]:
# считывание параметров обучения с файла config.yaml
det_epochs = config['detection_epochs']
det_logging = config['detection_logging']
log_interval = config["detection_log_wieghts_interval"]

# переменная с путем к изображению для "валидации"
test_img_path = ''

if det_logging: 
    # для логирования эксперимента понадобится файл с ключом к эксперименту из лк в comet-ml
    with open('secrets.json') as secrets_file:
        secrets = json.load(secrets_file)

    # init experimenxt
    experiment = Experiment(
        api_key=secrets["api_key"],
        project_name=secrets["project_name"],
        workspace="reu-ds-club", 
        tags=["detection"],
    )

    # считывание параметров обучения с файла config.yaml
    hyper_params = {
        "model_name": config["model"],
        "use_colab": config['use_colab'], 
        "epochs": det_epochs,
        "batch_size": config['batch_size'], 
        "image_size": config['img_size'], 
    }

    experiment.log_parameters(hyper_params)

for epoch in range(det_epochs):
    epoch_loss = 0.0
    for sample in (pbar := tqdm(train_detection_dataloader)):

        img, box = sample[0].to(device), sample[1].to(device)
        img = img.to(torch.float32)
        box = box.to(torch.float32)

        optimizer.zero_grad()
        pred = detection_model(img)
        loss = loss_fn(pred, box)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # уменьшение learning rate со временем
    scheduler.step()

    # Валидация модели на тестовом датасете
    val_loss = utils.validate_model(detection_model, test_detection_dataloader, loss_fn, device)

    print(f"Epoch: {epoch}\tLoss: {epoch_loss / len(train_detection_dataloader)}\tVal loss: {val_loss}")

    # выводить промежуточный результат работы нейронной сети с помощью загруженного изображения. 
    # Нужно лишь указать пусть к файлу .png в переменную test_img_path
    # Результат будет в папке results
    if test_img_path is not "":
        utils.save_img_after_epoch(test_img_path, detection_model, epoch, device)

    # Функция ранней остановки, если нейронная сеть перестает обучаться, то есть val_loss не падает 
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print("Early stopping")
        break

    # Сохранение весов модели после каждой эпохи
    checkpoint_filename = os.path.join(checkpoint_dir, f"{epoch}_checkpoint_detection.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': detection_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }, checkpoint_filename)

    #логирование метрики в comet-ml
    if det_logging:
        experiment.log_metric("loss", epoch_loss, step=epoch)
    
    # logging model weights (accorging to log_interval + last epoch)
    if det_logging and (epoch % log_interval == 0 or epoch == det_epochs-1):
        torch.save(detection_model, 'det_model.pth')
        experiment.log_model(name = f"model-epoch-{epoch}", file_or_folder = 'det_model.pth', file_name = f"det-model-epoch-{epoch}")
        experiment.log_asset(file_data = 'det_model.pth', file_name = f"det_model-epoch-{epoch}")
        print("save model")


if det_logging:
    experiment.end()

In [ ]:
# Setup for face recognition model training

# Устанавливаем скорость обучения (learning rate) для оптимизатора
lr = 1e-3

# Создаем модель студента для обучения (efficientnet_b1 с заменой классификатора)
student = timm.create_model('efficientnet_b1', pretrained=True)
student.classifier = nn.Linear(student.classifier.in_features, 512)
student = student.to(device)

# Создаем модель для триплет-обучения на основе студента
triplet_model = Triplet(student).to(device)

# Создаем модель учителя для распознавания лиц (InceptionResnetV1 с предобученными весами)
teacher = InceptionResnetV1(pretrained='vggface2').to(device)

# Задаем функцию потерь для обучения распознаванию лиц (Triplet Loss)
loss_for_recognition = TripletLoss(margin=5)

# Задаем оптимизатор (Adam) для обновления параметров модели
optimizer = torch.optim.Adam(triplet_model.parameters(), lr=lr)

# Задаем критерий для распознавания (Mean Squared Error Loss)
criterion = nn.MSELoss()

# Задаем шедулер для управления скоростью обучения (уменьшение скорости обучения на каждом шаге)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Задаем ранний остановщик для прекращения обучения, если потери не уменьшаются в течение некоторого времени
early_stopper = utils.EarlyStopping(patience=10, min_delta=0.01)

# Получаем загрузчик данных для обучения распознаванию лиц
recognition_dataloader = dataloaders.recognition_dataloader


In [ ]:
# считывание параметров обучения с файла config.yaml
rec_epochs = config['recognition_epochs']
rec_logging = config['recognition_logging']
rec_log_interval = config["recognition_log_wieghts_interval"]

if rec_logging: 
    # для логирования эксперимента понадобится файл с ключом к эксперименту из лк в comet-ml
    with open('secrets.json') as secrets_file:
        secrets = json.load(secrets_file)

    # init experimenxt
    experiment = Experiment(
        api_key=secrets["api_key"],
        project_name=secrets["project_name"],
        workspace="reu-ds-club", 
        tags=["recognition"],
    )

    # считывание параметров обучения с файла config.yaml
    hyper_params = {
        "model_name": config["model"],
        "use_colab": config['use_colab'], 
        "epochs": rec_epochs,
        "batch_size": config['batch_size'], 
        "image_size": config['img_size'], 
    }

    experiment.log_parameters(hyper_params)

for epoch in range(rec_epochs):
    epoch_loss = 0
    epoch_distilation_loss = 0
    for  triplet in (pbar := tqdm(recognition_dataloader)):

        anc, pos, neg = triplet

        preds = triplet_model(anc.to(device), pos.to(device), neg.to(device))
        
        triplet_loss = loss_for_recognition(*preds)

        triplet_encoder_output = triplet_model.encoder(anc.to(device))
        outputs_facenet = teacher(anc.to(device))
        distillation_loss = criterion(triplet_encoder_output, outputs_facenet)

        # get total loss
        total_loss = triplet_loss + distillation_loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        epoch_loss += total_loss.item()
        epoch_distilation_loss += distillation_loss.item()
    
    # уменьшение learning rate со временем
    scheduler.step()

    # Функция ранней остановки, если нейронная сеть перестает обучаться, то есть val_loss не падает 
    early_stopper(epoch_loss)
    if early_stopper.early_stop:
        print("Early stopping")
        break
    
    print(f'{epoch} | EPOCH LOSS: {total_loss} | DISTILATOIN LOSS: {epoch_distilation_loss}')
    
    # Сохранение весов модели после каждой эпохи
    checkpoint_filename = os.path.join(checkpoint_dir, f"{epoch}_checkpoint_recognition.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': triplet_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }, checkpoint_filename)

    #логирование метрики в comet-ml
    if rec_logging:
        experiment.log_metric("loss", epoch_loss, step=epoch)
    
    # logging model weights (accorging to log_interval + last epoch)
    if rec_logging and (epoch % log_interval == 0 or epoch == rec_epochs-1):
        torch.save(triplet_model, 'rec_model.pth')
        experiment.log_model(name = f"rec_model-epoch-{epoch}", file_or_folder = 'rec_model.pth', file_name = f"rec_model-epoch-{epoch}")
        experiment.log_asset(file_data = 'rec_model.pth', file_name = f"rec_model-epoch-{epoch}")
        print("save model")


if rec_logging:
    experiment.end()